# Логика

In [ ]:
import numpy as np
import torch

In [ ]:
import sys
sys.path.insert(0, './python')

In [ ]:
from fuzzy_torch import logic

In [ ]:
# Логика.
#Logic = logic.Godel
#Logic = logic.Product
#Logic = logic.Lukasiewicz
#Logic = logic.Nilpotent
Logic = logic.Hamacher

# Алиасы для оераций.
fzand  = Logic.fuzzy_and
fzor   = Logic.fuzzy_or
fzimpl = Logic.fuzzy_impl

## Градиенты

In [ ]:
a = torch.tensor(np.array([0.5, 0.9]), requires_grad=True)
b = torch.tensor(np.array([0.7, 0.8]), requires_grad=True)

In [ ]:
d = torch.sum(logic.Product.fuzzy_or(a, b))

In [ ]:
d.backward()

In [ ]:
b.grad

## Индикаторные функции

In [ ]:
from fuzzy_torch.modules.indicators import * #Sigmoid, AbsSigmoid, Gaussian

In [ ]:
params = Sigmoid((2))

In [ ]:
params.linear.weight = torch.nn.Parameter(torch.tensor([[0.5, 0.5]]))
params.linear.bias = torch.nn.Parameter(torch.tensor([-0.5]))

In [ ]:
print(params.linear.weight)
print(params.linear.bias)

In [ ]:
params(torch.log(torch.tensor([[0.0000001, 1.0], [200.0, 0.1]])))

In [ ]:
(torch.zeros(2) + 0.0001) / (torch.zeros(2) + 0.0001)

## Простейшие логические выражения

In [ ]:
N_temp = 101
N_wind = 101

temperature_np = np.linspace(-40.0, 60.0, N_temp)
wind_speed_np = np.linspace(0.0, 10, N_wind)

# Сетка значений.
temperature_np_grid, wind_speed_np_grid = np.meshgrid(temperature_np, wind_speed_np)
temperature_np_grid = temperature_np_grid.flatten()[:,None].astype(np.float32)
wind_speed_np_grid = wind_speed_np_grid.flatten()[:,None].astype(np.float32)

In [ ]:
temperature = torch.tensor(temperature_np_grid, requires_grad=True)
wind_speed  = torch.tensor(wind_speed_np_grid, requires_grad=True)

In [ ]:
class WindowController(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # Логические переменные.
        self.wind_speed_is_high  = Sigmoid(1, weight=2.0, offset=np.log10(7.5))
        self.temperature_is_low  = Sigmoid(1, weight=-0.01, offset=-5.0)
        self.temperature_is_high = Sigmoid(1, weight=0.01, offset=35.0)
        
    def forward(self, x):
        # Логическое выражение.
        result = fzor(
            self.wind_speed_is_high(torch.log10(x[:,1,None] + 1e-7)),
            fzor(
                self.temperature_is_low(x[:,0,None]),
                self.temperature_is_high(x[:,0,None])
            )
        )
        
        return result

In [ ]:
controller = WindowController()

In [ ]:
grid_X = torch.cat((temperature, wind_speed), dim=1)

In [ ]:
grid_Y = controller(grid_X).detach().numpy()[:,0].reshape((N_temp, N_wind))

### Обучение

In [ ]:
# Обучающая выборка
train_X = np.array([
    [-9.0, 1.0],
    [-39.3, 3.0],
    [-1.0, 2.2],
    [9.0, 5.0],
    [13.0, 9.0],
    [-2.5, 4.0],
    [32.0, 2.0],
    [40.0, 3.0],
    [25.6, 2.0],
    [22.3, 5.0],
    [31.0, 4.0],
    [37.0, 0.1],
    [43.0, 0.0],
    [21.4, 7.0],
    [19.0, 6.7],
    [27.6, 7.4],
    [25.2, 1.5],
    [-19.5, 0.6],
    [-18.5, 2.2],
    [19.4, 0.4],
    [15.8, 1.2],
]).astype(np.float32)

train_y = np.array([
    0.0,
    1.0,
    0.0,
    0.0,
    1.0,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
    1.0,
    0.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
]).astype(np.float32)[:,None]

In [ ]:
import matplotlib.pyplot as pl

fig = pl.figure()
ax = fig.gca()
cfset = ax.contourf(temperature_np, wind_speed_np, grid_Y, cmap='Blues', levels=100)
ax.scatter(train_X[:,0], train_X[:,1], c=train_y, cmap='Blues')
ax.set_xlabel('temp')
ax.set_ylabel('wind')

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_X), torch.tensor(train_y))

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(controller.parameters(), lr=0.05)#, momentum=0.9)

In [ ]:
loss = torch.nn.BCELoss()

for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        x, true_y = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        y = controller(x)
        eval_loss = loss(y, true_y)
        eval_loss.backward()
        optimizer.step()

        # print statistics
        running_loss += eval_loss.item()
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss:.3f}')
        running_loss = 0.0

print('Finished Training')

In [ ]:
grid_Y = controller(grid_X).detach().numpy()[:,0].reshape((N_temp, N_wind))

In [ ]:
import matplotlib.pyplot as pl

fig = pl.figure()
ax = fig.gca()
cfset = ax.contourf(temperature_np, wind_speed_np, grid_Y, cmap='Blues', levels=100)
ax.scatter(train_X[:,0], train_X[:,1], c=train_y, cmap='Blues')
ax.set_xlabel('temp')
ax.set_ylabel('wind')

## Привет Студсовету ФПМИ

In [ ]:
class WashingMachineController(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # Логические переменные.
        self.D_is_high    = Triangle(1, weight=(1/50), offset=100)
        self.D_is_medium  = Triangle(1, weight=(1/50), offset=0)
        self.D_is_low     = Triangle(1, weight=(1/50), offset=50)
        self.G_is_medium  = Triangle(1, weight=(1/25), offset=25.0)
        self.G_is_high    = Triangle(1, weight=(1/100), offset=100.0)
        self.G_is_none    = Singletone()
        
    def forward(self, D, G):
        # Логическое выражение.
        #print(self.D_is_low(D).detach().numpy(), torch.eq(G,0).detach().numpy())
        VS = fzand(self.D_is_low(D), self.G_is_none(G))#torch.eq(G,0))
        S  = fzand(self.D_is_medium(D), self.G_is_none(G))#torch.eq(G,0))
        N  = fzor(fzand(self.D_is_high(D), self.G_is_none(G)),#torch.eq(G,0)),
                  fzand(self.D_is_low(D), self.G_is_medium(G)),
                  fzand(self.D_is_medium(D), self.G_is_medium(G)))
        L  = fzor(fzand(self.D_is_high(D), self.G_is_medium(G)),
                  fzand(self.D_is_low(D), self.G_is_high(G)),
                  fzand(self.D_is_medium(D), self.G_is_high(G)))
        VL = fzand(self.D_is_high(D), self.G_is_high(G))
        
        result = (30 * VS + 45 * S + 90 * N + 150 * L + 210 * VL) / (VS + S + N + L + VL)
        
        return result

In [ ]:
washing_machine_1 = WashingMachineController()

In [ ]:
D = torch.tensor(np.array([[100]]).astype(np.float32))
G = torch.tensor(np.array([[100]]).astype(np.float32))

print(washing_machine_1(D, G).detach().numpy()[0,0])

In [ ]:
N_D = 101
N_G = 101

D_np = np.linspace(0.0, 100.0, N_D)
G_np = np.linspace(0.0, 100.0, N_G)

# Сетка значений.
D_np_grid, G_np_grid = np.meshgrid(D_np, G_np)
D_np_grid = D_np_grid.flatten()[:,None].astype(np.float32)
G_np_grid = G_np_grid.flatten()[:,None].astype(np.float32)

In [ ]:
D = torch.tensor(D_np_grid, requires_grad=True)
G = torch.tensor(G_np_grid, requires_grad=True)

In [ ]:
grid_Y = washing_machine_1(D,G).detach().numpy()[:,0].reshape((N_D, N_G))

In [ ]:
import matplotlib.pyplot as pl

fig = pl.figure()
ax = fig.gca()
cfset = ax.contourf(D_np, G_np, grid_Y, cmap='Blues', levels=100)
ax.set_xlabel('D, %')
ax.set_ylabel('G, %')

## Бинарные отношения

In [ ]:
# Логика.
Logic = logic.Godel
#Logic = logic.Product
#Logic = logic.Lukasiewicz
#Logic = logic.Nilpotent
#Logic = logic.Hamacher

In [ ]:
relation_A = torch.tensor(np.array([[0.1, 0.4],
                                    [0.7, 0.5]]).astype(np.float32))[None,:]
relation_B = torch.tensor(np.array([[0.8, 0.7, 0.6],
                                    [0.2, 0.4, 0.1]]).astype(np.float32))[None,:]

In [ ]:
def compose(relation_A, relation_B):
    composition = torch.zeros(relation_A.size()[0], relation_A.size()[1], relation_B.size()[2])
    
    for a_idx in range(relation_A.size()[1]):
        for b_idx in range(relation_B.size()[2]):
            value = torch.zeros(relation_A.size()[0])
            
            for c_idx in range(relation_A.size()[2]):
                value = Logic.fuzzy_or(value, Logic.fuzzy_and(relation_A[:, a_idx, c_idx],
                                                              relation_B[:, c_idx, b_idx]))
            
            composition[:, a_idx, b_idx] = value
            
    return composition

In [ ]:
compose(relation_A, relation_B)

In [ ]:
x_y = torch.tensor(np.array([[[0.9,0.9,0.8,0.4,0.5],
                              [0.8,0.5,0.9,0.3,0.1],
                              [0.3,0.9,0.6,0.5,0.9],
                              [0.5,0.4,0.5,0.5,0.2],
                              [0.7,0.8,0.8,0.2,0.6]]]).astype(np.float32))

y_z = torch.tensor(np.array([[[0.9,0.8,0.7,0.9,1.0],
                              [0.6,0.4,0.8,0.5,0.6],
                              [0.5,0.2,0.3,0.8,0.7],
                              [0.5,0.9,0.5,0.8,0.4],
                              [1.0,0.6,0.5,0.7,0.4],
                              [0.4,0.5,1.0,0.7,0.8],
                              [0.5,0.8,0.9,0.5,0.4],
                              [0.5,0.6,0.7,0.6,0.5],
                              [0.8,1.0,0.2,0.5,0.6],
                              [0.3,0.5,0.9,0.6,0.8]]]).astype(np.float32))

x_z = compose(x_y, y_z)
print(x_z)